### Span Detection Component ###

In [1]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import utils.pdf_reader as pdfr

2023-10-10 20:17:59.848387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 20:18:01.557717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [5]:
input_prompt = pdfr.tostring_from_pdf("https://arxiv.org/pdf/1706.03762.pdf", "tester.pdf")
input_prompt_words = input_prompt.split(" ")
input_question = "Summarize the following article.".split(" ")
segments = []
for i in range(0, len(input_prompt_words), 512 - len(input_question)):
    if i + 512 - len(input_question) >= len(input_prompt_words):
        curr_seg = input_question + input_prompt_words[i:len(input_prompt_words) - len(input_question)]
    else:
        curr_seg = ' '.join(input_question + input_prompt_words[i:i + 512 - len(input_question)])
    segments.append(curr_seg)
print(segments)

PdfReadError: EOF marker not found

In [20]:
concatenated = ""
for segment in segments: 
    input = tokenizer.encode(segment, return_tensors="pt").to("cuda")
    gen_tokens = model.generate(
        input,
        do_sample=True,
        temperature=0.9,
        max_length=700,
    )
    concatenated += gen_tokens[len(input):] + " "

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


/home/vijays2/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 3587, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [156,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [156,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [156,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [156,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [156,0,0], thread: [68,0,0]

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
gen_text

'In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.\n\nThe findings were published in the journal Current Biology. The paper includes a very interesting quote from a scientist that was published in the journal:\n\n“All of our observations suggest that the region may well be suitable for reintroducing the endangered red-throated humming'